In [99]:
import cv2
import numpy as np

In [100]:
img = cv2.imread('02_image.jpg')
img_grqy = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# 模糊處理
img_blur = cv2.GaussianBlur(img_grqy,(7,7),0)
# 邊緣檢測
edges = cv2.Canny(img_blur,100,370)
cv2.imwrite('canny_edges.jpg', edges)

True

In [101]:
rho = 1    # 距離解析度
theta = np.pi / 170 # 角度解析度
threshold = 100   # 閾值
min_line_length = 400 # 最短線段長度
max_line_gap = 90 # 最大線段間隙
lines = cv2.HoughLinesP(edges, rho, theta, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)
result = img.copy()
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(result, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imwrite('hough_lines_result.jpg', result)

True

In [102]:
loc = []
for line in lines:
    x1,y1,x2,y2 = line[0]
    loc.extend([[x1,y1],[x2,y2]])
loc = np.array(loc,dtype=np.float32)


In [103]:
#kmeans分群
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4
_, labels, centers = cv2.kmeans(loc, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)

# 轉回整數
final_corners = centers.astype(np.int32)

final_corners_y_sort = np.argsort(final_corners[:,1])
top = [final_corners[i] for i in final_corners_y_sort[:2]]
bottom = [final_corners[i] for i in final_corners_y_sort[-2:]]

if top[0][0] > top[1][0]:
    top_left = top[1]
    top_right = top[0]
else:
    top_left = top[0]
    top_right = top[1]
    
if bottom[0][0] > bottom[1][0]:
    bottom_left = bottom[1]
    bottom_right = bottom[0]
else:
    bottom_left = bottom[0]
    bottom_right = bottom[1]

square = np.array([top_left,top_right,bottom_right,bottom_left])

In [106]:
final = img.copy()
for i in range(4):
    cv2.line(final,tuple(square[i]),tuple(square[(i+1)%4]),(0,255,0),2)
cv2.imwrite('hough_lines_result_with_boundary.jpg', final)


True